In [1]:
# -*- coding: utf-8 -*-

import os, sys, csv, re, urllib.request
from bs4 import BeautifulSoup  #<---- Need to install this package manually using pip

#os.chdir("/content/gdrive/My Drive/Colab Notebooks/SP DATA")#<===The location of your csv files
htmlSubPath = 'HTML2' #<===The subfolder with the 10-K files in HTML format
txtSubPath = 'txt' #<===The subfolder with the extracted text files

DownloadLogFile = "10kDownloadLog.csv" #a csv file (output of the 3DownloadHTML.py script) with the download history of 10-K forms
ReadLogFile = "10kReadlog.csv" #a csv file (output of the current script) showing whether item 1 is successfully extracted from 10-K forms

"""
BeginSuccessHashLinksFile = "BeginSuccessHashLinks.csv"
BeginFailHashLinksFile = "BeginFailHashLinks.csv"


EndSuccessHashLinksFile = "EndSuccessHashLinks.csv"
EndFailHashLinksFile = "EndFailHashLinks.csv"
"""


#broken_pages = ["WU_"]

#list of unformatted/broken pages to remove from the search. this is no longer necessary, the algorithm discards these now.
broken_pages = []


# check if the link has a corresponding html element on the page, if not return -1
# and discard this match
def getIndex(page,match):
    index = page.find("name=\""+str(match.group(1)[1:])+"\"")
    if(index)<0:
        index = page.find("name='"+str(match.group(1)[1:])+"'")
        if(index)<0:
            return -1

    return index




def readAsPlainFile(file_name):

    input_path = os.path.join(os.getcwd(),htmlSubPath, file_name)
    output_path = os.path.join(os.getcwd(),txtSubPath, file_name.replace(".htm",".txt"))

    #print("readAsPlainFile is trying "+input_path)

    #regex = r'(risk factors|item.?1a)(?:(?!\1).){100,}?(?=unresolved.?staff|item.?1b|item.?2)'
    regex = r'(?:risk factors|item.?1\(?a)(?:(?!(risk factors|item.?1\(?a)).){200,}?(?=unresolved.?staff|item.?1\(?b)|item.?2\(?b'

    input_file = open(input_path,'r')
    #print(input_file)
    page = input_file.read()  #<===Read the HTML file into Python

    page = re.sub('<[^>]*>', ' ',page)


    page = re.sub(r'&#?\w{0,5};',' ',page,flags=re.IGNORECASE)

    page = page.strip()  #<=== remove white space at the beginning and end
    page = page.replace('\n', ' ') #<===replace the \n (new line) character with space
    page = page.replace('\r', ' ') #<===replace the \r (carriage returns -if you're on windows) with space
    page = page.replace('\t', ' ') #<===replace the \t tab
    """
    page = page.replace('&nbsp;', ' ') #<===replace "&nbsp;" (a special character for space in HTML) with space. 
    page = page.replace('&#160;', ' ') #<===replace "&#160;" (a special character for space in HTML) with space.
    page = page.replace('&#32;', ' ') #<===replace "&#160;" (a special character for space in HTML) with space.
    page = page.replace('&#8211;', ' ') #<===replace "&#160;" (a special character for space in HTML) with space.
    """
    
    while '  ' in page:
        page = page.replace('  ', ' ') #<===remove extra space
    #print(page)

    match = re.search(regex,page,flags=re.IGNORECASE)

    if(match):
        #print("Success! "+output_path)
        output_file = open(output_path, "w", encoding='utf-8')
        text_with_prefix = match.group(0)
        text_without_prefix = re.sub(r'(risk factors|item.?1\(?a)',' ',text_with_prefix[:40],flags=re.IGNORECASE) +text_with_prefix[40:] 

        # remove "table of contents" string from the text
        text_without_TOC = re.sub(r'table of contents',' ',text_without_prefix,flags=re.IGNORECASE) 

        output_file.write(text_without_TOC)  
        output_file.close()
        return match
    else:
        #print("fail "+output_path)
        #print("__")
        return None
    input_file.close()



def readAsHTML(file_name):
    input_path = os.path.join(os.getcwd(),htmlSubPath, file_name)
    output_path = os.path.join(os.getcwd(),txtSubPath, file_name.replace(".htm",".txt"))
    

    input_file = open(input_path,'r')
    print(input_file)
    page = input_file.read()  #<===Read the HTML file into Python


    #Pre-processing the html content by removing extra white space and combining then into one line.
    page = page.strip()  #<=== remove white space at the beginning and end
    page = page.replace('\n', ' ') #<===replace the \n (new line) character with space
    page = page.replace('\r', ' ') #<===replace the \r (carriage returns -if you're on windows) with space
    page = page.replace('&nbsp;', ' ') #<===replace "&nbsp;" (a special character for space in HTML) with space. 
    page = page.replace('&#160;', ' ') #<===replace "&#160;" (a special character for space in HTML) with space.
    page = page.replace('&#32;', ' ') #<===replace "&#160;" (a special character for space in HTML) with space.
    page = page.replace('&#8211;', ' ') #<===replace "&#160;" (a special character for space in HTML) with space.

    
    while '  ' in page:
        page = page.replace('  ', ' ') #<===remove extra space

    page = page.lower()
    #Using regular expression to extract texts that match a pattern 

    #debug
    """
    output_file = open("dummy_debug.txt", "w", encoding='utf-8')
    output_file.write(page)  
    output_file.close()
    """

    #some pages don't have item 1a, check if the page is formatted correctly.
    regex_1a = r'\b1a\b'
    match = re.search(regex_1a,page, flags=re.IGNORECASE)

    if(not match):
        input_file.close()    

        return None

 

    #2 regexes to find links (<a> element with href) for elements containing titles  "item 1a", or "risk factors"
    beginRegexs = [r'>[\s|\.]{0,2}(?:item \b1a\b|\bitem1a\b|\bitem-1a\b|\b1a\b|\brisk\b).{0,600}?href=[\'\"](#[a-z0-9_.]+)[\'\"]',
            r'href=[\'\"](#[a-z0-9_.]+)[\'\"](?:(?!href).){0,600}?>[\s|\.]{0,2}(?:item \b1a\b|\bitem1a\b|\bitem-1a\b|\b1a\b|\brisk\b)']

    #2 regexes to find links (<a> element with href) for elements containing titles "item 1b", or "unresolved"
    endRegexs = [r'>[\s|\.]{0,2}(?:item \b1b\b|\bitem1b\b|\bitem-1b\b|\b1b\b|\bunresolved\b).{0,600}?href=[\'\"](#[a-z0-9_.]+)[\'\"]',
            r'href=[\'\"](#[a-z0-9_.]+)[\'\"](?:(?!href).){0,600}?>[\s|\.]{0,2}(?:item \b1b\b|\bitem1b\b|\bitem-1b\b|\b1b\b|\bunresolved\b)']
    
    #Now we try to see if a match can be found...

    #list of matches to check if they are correctly extracted.    
    matchesBeginList = []
    matchesEndList = []


    for beginReg in beginRegexs:


        matches = re.finditer (beginReg, page, flags=re.IGNORECASE)
        #If a match exist....
        for matchBegin in matches:
            if matchBegin:

                #check if the regex has found the hashtag of "table of contents" #toc, if so just discard this result.
                if(matchBegin.group(1)=="#toc"):
                    continue

                # find the element
                beginIndex = getIndex(page,matchBegin)

                # if no result is fou
                if(beginIndex<0):
                    continue

                #check if found regex is at a location near the title containing (risk|factor|item.?1a)
                correctPassageMatch = re.search(r'(risk|factor|item.?1a)',
                                                page[beginIndex-10:beginIndex+1000],
                                                flags=re.IGNORECASE)

                #if not, discrad this result
                if(not correctPassageMatch):
                    #print("\n")
                    #print(page[beginIndex-50:beginIndex+1000])
                    continue

                #add the match to check it after the loop
                matchesBeginList.append(matchBegin)


    
    #get the match closest to the "item 1a", "risk factors" titles
    if(len(matchesBeginList)>0):
        matchesBeginList.sort(key = lambda x:len(x.group(0)))

        matchBegin = matchesBeginList[0]


        beginIndex = getIndex(page,matchBegin)
        if(beginIndex<0):
            input_file.close()    
            return None       


        
        #print(page[beginIndex-5:beginIndex+1000])
        #print(matchBegin.group(1))

    else:
        input_file.close()    
        return None



    #same steps as above for the ending node of the passage
    for endReg in endRegexs:

        matches = re.finditer (endReg, page, flags=re.IGNORECASE)

        for matchEnd in matches:

            if matchEnd:

                if(matchEnd.group(1)=="#toc"):
                    continue

                #for debugging    
                """
                print(matchEnd.group(1))
                print(matchEnd.start())

                #soup = BeautifulSoup(matchBegin.group(1), "html.parser") #<=== match.group(1) returns the texts inside the parentheses (.+?) 

                print(soup.find('a', {'name': matchEnd.group(1)[1:]}))
                """
                #print(soup.text)
                #print(matchEnd.group(1))
                endIndex = getIndex(page,matchEnd)

                if(endIndex<0):
                    continue
                
                
                #check if found regex is at a location near the title containing (unresolved|item.?1b)
                correctPassageMatch = re.search(r'(unresolved|item.?1b)',
                                                page[endIndex-10:endIndex+1000],
                                                flags=re.IGNORECASE)

                if(not correctPassageMatch):
                    #print("\n")
                    #print(page[endIndex-50:endIndex+700])
                    continue
                matchesEndList.append(matchEnd)


                

    if(len(matchesEndList)>0):
        matchesEndList.sort(key = lambda x:len(x.group(0)))

        matchEnd = matchesEndList[0]


        endIndex = getIndex(page,matchEnd)
        if(endIndex<0):
            input_file.close()    
            return None

    else:
        input_file.close()    
        return None


    # if any index is not found, ignore this page.
    # failed extraction.
    if(beginIndex<0 or endIndex<0):
        #print(matchEnd.group(1)[1:])
        #print("fail broken_page "+file_name)
        input_file.close()
        return None
        #print(page[beginIndex:endIndex])



    #for debugging    

    """
    dummy_text =  page[beginIndex-3:endIndex-3]
    dummy_text = re.sub('<[^>]*>', ' ',dummy_text)

    while '  ' in dummy_text:
        dummy_text = dummy_text.replace('  ', ' ')

    print("################\n\n")
    print(dummy_text)


    print("################\n\n")
    """

    # removes the html tags and only keep the texts
    soup = BeautifulSoup(page[beginIndex-3:endIndex-3], "html.parser")

    output_file = open(output_path, "w", encoding='utf-8')

    text_with_prefix = soup.get_text()
    #text_without_prefix = re.sub(r'(risk factors|item.?1\(?a)',' ',text_with_prefix[:40],flags=re.IGNORECASE)
    #text_without_prefix = re.sub(r'table of contents',' ',text_without_prefix,flags=re.IGNORECASE)


    #text_with_prefix = match.group(0)
    #remove the initial "risk factors" or "item 1a" strings from the text
    text_without_prefix = re.sub(r'(risk factors|item.?1\(?a)',' ',text_with_prefix[:40],flags=re.IGNORECASE) +text_with_prefix[40:] 

    # remove "table of contents" string from the text
    text_without_TOC = re.sub(r'table of contents',' ',text_without_prefix,flags=re.IGNORECASE) 

    output_file.write(text_without_TOC) 

    #output_file.write(text_with_prefix)  


    #output_file.write(soup.get_text())  
    output_file.close()
    
    #print(" found Match!\n")
    #foundMatch = True
            


    #remove space at the beginning and end and the subtitle "business" at the beginning
    #^ matches the beginning of the text
    #outText = re.sub("^business\s*","",rawText.strip(),flags=re.IGNORECASE)


    #for debugging    
    """
    output_file = open(output_path, "w", encoding='utf-8')
    output_file.write(rawText)  
    output_file.close()
    """


    input_file.close()    

    return match

def main():

    if not os.path.isdir(txtSubPath):  ### <=== keep all texts files in this subfolder
        os.makedirs(txtSubPath)
        
    csvFile = open(DownloadLogFile, "r") #<===A csv file with the list of 10k file names (the file should have no header)
    csvReader = csv.reader(csvFile,delimiter=",")
    csvData = list(csvReader)
    
    logFile = open(ReadLogFile, "w") #<===A log file to track which file is successfully extracted
    logWriter = csv.writer(logFile, quoting = csv.QUOTE_NONNUMERIC)

    #for debugging    
    """
    BeginSuccessHashLinksCSV = open(BeginSuccessHashLinksFile, "w") #<===A log file to track beginning hash links
    beginSuccessHashLogWriter = csv.writer(BeginSuccessHashLinksCSV, quoting = csv.QUOTE_NONNUMERIC)

    BeginFailHashLinksCSV = open(BeginFailHashLinksFile, "w") #<===A log file to track beginning hash links
    beginFailHashLogWriter = csv.writer(BeginFailHashLinksCSV, quoting = csv.QUOTE_NONNUMERIC)
    
    EndSuccessHashLinksCSV = open(EndSuccessHashLinksFile, "w") #<===A log file to track beginning hash links
    endSuccessHashLogWriter = csv.writer(EndSuccessHashLinksCSV, quoting = csv.QUOTE_NONNUMERIC)

    EndFailHashLinksCSV = open(EndFailHashLinksFile, "w") #<===A log file to track beginning hash links
    endFailHashLogWriter = csv.writer(EndFailHashLinksCSV, quoting = csv.QUOTE_NONNUMERIC)

    beginSuccessHashLogWriter.writerow(["link","hash"])
    beginFailHashLogWriter.writerow(["link","hash"])

    endSuccessHashLogWriter.writerow(["link","hash"])
    endFailHashLogWriter.writerow(["link","hash"])

    EndSuccessHashLinksCSV.close()
    EndFailHashLinksCSV.close()


    BeginSuccessHashLinksCSV.close()
    BeginFailHashLinksCSV.close()
    """

    logWriter.writerow(["filename","extracted"])

    logFile.close()





    i=1
    fail = 0
    for rowData in csvData[1:]:
        if(len(rowData)==0):
            continue
##        Ticker = rowData[0]
##        DocIndex = rowData[1]
##        IndexLink = rowData[2]
##        Description = rowData[3]
##        FilingDate = rowData[4]
##        NewFilingDate = rowData[5]
##        FormLink = rowData[6]
##        FormName = rowData[7]
        FileName = rowData[8]
        #print(i)
        
        #for debugging
        """
        if(FileName[0:3]!="F_7"):
            continue
        """
        if(FileName in broken_pages):
            print("Fail!\n")

            logFile = open(ReadLogFile, "a") #<===A log file to track which file is successfully extracted

            logWriter = csv.writer(logFile, quoting = csv.QUOTE_NONNUMERIC)
            logWriter.writerow([FileName,"no"])
            logFile.close()

            continue
        


        if ".htm" in FileName:        
            match=readAsHTML(FileName)
            #match = returnedValues[0]
            
            #for debugging
            """
            if(FileName=="ZTS_4_2017_02_16.htm"):
                print(FileName)
            else:
                print(FileName)
            """


            logFile = open(ReadLogFile, "a") #<===A log file to track which file is successfully extracted

            logWriter = csv.writer(logFile, quoting = csv.QUOTE_NONNUMERIC)

            if match:
                print("HTML Success!\n")

                
                #for debugging

                """
                BeginSuccessHashLinksCSV = open(BeginSuccessHashLinksFile, "a") #<===A log file to track beginning hash links
                beginSuccessHashLogWriter = csv.writer(BeginSuccessHashLinksCSV, quoting = csv.QUOTE_NONNUMERIC)
                #beginSuccessHashLogWriter.writerow([returnedValues[1].name,match.group(1)])
                BeginSuccessHashLinksCSV.close()
                """
                logWriter.writerow([FileName,"yes"])
                

            else:

                match=readAsPlainFile(FileName)
                if(match):
                    print("PlainFile Success!\n")
                    logWriter.writerow([FileName,"yes"])

                else:
                    fail+=1
                    print("Fail! \n"+str(fail)+"/"+str(i) +" "+ FileName)

                    #for debugging
                    """
                    BeginFailHashLinksCSV = open(BeginFailHashLinksFile, "a") #<===A log file to track beginning hash links
                    beginFailHashLogWriter = csv.writer(BeginFailHashLinksCSV, quoting = csv.QUOTE_NONNUMERIC)
                    beginFailHashLogWriter.writerow([returnedValues[1].name,"None"])
                    BeginFailHashLinksCSV.close()
                    """
                    logWriter.writerow([FileName,"no"])

            logFile.close()
        i=i+1
        

    print("done!")

    #for debugging
    """
    BeginSuccessHashLinksCSV.close()
    BeginFailHashLinksCSV.close()
    """
    logFile.close()     


if __name__ == "__main__":
    main()


<_io.TextIOWrapper name='/Users/hilal.pataci/Desktop/sec_cleaning/HTML2/A_1_2019_12_19.htm' mode='r' encoding='UTF-8'>
Fail! 
1/1 A_1_2019_12_19.htm
<_io.TextIOWrapper name='/Users/hilal.pataci/Desktop/sec_cleaning/HTML2/A_2_2018_12_20.htm' mode='r' encoding='UTF-8'>
HTML Success!

<_io.TextIOWrapper name='/Users/hilal.pataci/Desktop/sec_cleaning/HTML2/A_3_2017_12_21.htm' mode='r' encoding='UTF-8'>
HTML Success!

<_io.TextIOWrapper name='/Users/hilal.pataci/Desktop/sec_cleaning/HTML2/A_4_2016_12_20.htm' mode='r' encoding='UTF-8'>
HTML Success!

<_io.TextIOWrapper name='/Users/hilal.pataci/Desktop/sec_cleaning/HTML2/A_5_2015_12_21.htm' mode='r' encoding='UTF-8'>
HTML Success!

<_io.TextIOWrapper name='/Users/hilal.pataci/Desktop/sec_cleaning/HTML2/A_6_2014_12_22.htm' mode='r' encoding='UTF-8'>
HTML Success!

<_io.TextIOWrapper name='/Users/hilal.pataci/Desktop/sec_cleaning/HTML2/A_7_2013_12_19.htm' mode='r' encoding='UTF-8'>
HTML Success!

<_io.TextIOWrapper name='/Users/hilal.pataci/De